In [ ]:
# %pip install gym_super_mario_bros==7.3.0 nes_py

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
#import the game
import gym_super_mario_bros
#import the joypad wrapper
from nes_py.wrappers import JoypadSpace
#import the simplified controls
from gym_super_mario_bros.actions import SIMPLE_MOVEMENT

In [ ]:
import gym

class CustomJoypadSpace(gym.Wrapper):
    def __init__(self, env):
        super().__init__(env)

    def reset(self, **kwargs):
        return super().reset()

In [ ]:
# %pip install torch torchvision torchaudio

In [ ]:
# %pip install stable-baselines3[extra]

In [ ]:
#import framestacker and grayscaler
from gym.wrappers import GrayScaleObservation
#import vectorization wrappers
from stable_baselines3.common.vec_env import VecFrameStack,DummyVecEnv
#import matplotlib to show framestacking
from matplotlib import pyplot as plt

In [ ]:
# JoypadSpace.reset = lambda self, **kwargs: self.env.reset(**kwargs)

#1.Create base environment
env=gym_super_mario_bros.make('SuperMarioBros-v0',apply_api_compatibility=True,render_mode='human')
#2.Simplify Controls
env=JoypadSpace(env,SIMPLE_MOVEMENT)
#3.Grayscale
env=GrayScaleObservation(env,keep_dim=True)
#4.Wrap inside Dummy env
env=CustomJoypadSpace(env)
env=DummyVecEnv([lambda:env])
#5.Stack the frames
env=VecFrameStack(env,4,channels_order='last')

In [ ]:
# state=env.reset()

In [ ]:
# state,reward,done,info=env.step([env.action_space.sample()])
# plt.imshow(state[0])

In [ ]:
#File Path Management
import os
#PPO Algo
from stable_baselines3 import PPO
#Base Callback for saving model
from stable_baselines3.common.callbacks import BaseCallback

In [ ]:
class TrainAndLoggingCallback(BaseCallback):
    def __init__(self,check_freq,save_path,verobose=1):
        super(TrainAndLoggingCallback,self).__init__(verobose)
        self.check_freq = check_freq
        self.save_path=save_path

    def _init_callback(self):
        if self.save_path is not None:
            os.makedirs(self.save_path,exist_ok=True)

    def _on_step(self):
        if self.n_calls%self.check_freq==0:
            model_path=os.path.join(self.save_path,'best_model_{}'.format(self.n_calls))
            self.model.save(model_path)
        return True

In [ ]:
CHECKPOINT_DIR='./train/'
LOG_DIR='./logs/'

In [ ]:
#Model saving callback
callback=TrainAndLoggingCallback(check_freq=100000,save_path=CHECKPOINT_DIR)

In [ ]:
#Untrained AI model
# model=PPO('CnnPolicy',env,verbose=1,tensorboard_log=LOG_DIR,learning_rate=0.000001,n_steps=512)

In [ ]:
#Train model
# model.learn(total_timesteps=100000,callback=callback)

In [ ]:
# model.save('./train/best_model_100000')

In [ ]:
#Load model
model=PPO.load('./train/best_model_100000')

In [ ]:
#Start game
state=env.reset()
#Loop through predicted actions
while True:
    action,_=model.predict(state)
    state,reward,done,info=env.step(action)
    env.render()

In [ ]:
env.close()